In [1]:
from bx.intervals.intersection import Intersecter, Interval

In [15]:
expte = set([])
te = open("/home/zliang/Documents/projects-in-Nathan-lab/stress-experiment/ca20a3/ca20a3-B73v4-W22-Mo17index/2020JuneReanalysis/CPMvalue/fourcategories-TEelement-expression/B73-control-expressed.txt", "r")
te.readline()
for x in te:
    y = x.strip()
    expte.add(y)
te.close()

In [16]:
teann = open("/home/zliang/Documents/genomeinfo/B73.v4.41.structuralTEv2.2020.03.22.filteredTE.disjoined.gff3",'r')
expte_dict = {}

good_chrom = set([])
for x in range(1, 11):
    y = str(x)
    good_chrom.add(y)
    expte_dict[y] = Intersecter()

mylen = 0

for line in teann:
    new = line.strip().split('\t')
    name = new[-1].split(';')[0].replace('ID=','').replace('gene:','')
    st = int(new[3])
    sp = int(new[4])
    mychr = new[0].replace('chr','')
    diff = abs(st-sp)+1
    if name in expte:
        mylen += diff
        expte_dict[mychr].add_interval(Interval(st, sp))

teann.close()

print (mylen)

45725052


In [17]:
umr = open("/home/zliang/Documents/genomeinfo/UMRdata/B73.control.UMRs.bed","r")

umr_dict = {}

good_chrom = set([])

for x in range(1, 11):
    y = str(x)
    good_chrom.add(y)
    umr_dict[y] = Intersecter()
    
for line in umr:
    new = line.strip().split('\t')
    mychr = new[0].replace('chr','')
#    mychr = str(int(new[0].replace('M',''))) # for Mo17
    st = int(new[1])
    sp = int(new[2])
    diff = abs(st-sp)+1
    if mychr not in good_chrom:continue
    umr_dict[mychr].add_interval(Interval(st, sp, value=diff))
    
umr.close()

In [18]:
umr = open("/home/zliang/Documents/genomeinfo/UMRdata/B73.control.UMRs.bed","r")

alist = ["-1000","-900","-800","-700","-600","-500","-400","-300","-200","-100","UMR",
        "100","200","300","400","500","600","700","800","900","1000"]

out = open("B73-control-expressed-UMR-regions.bed",'w')

mylen = 0
xx = 1
for line in umr:
    new = line.strip().split('\t')
    mychr = new[0].replace('chr','')
#    mychr = str(int(new[0].replace('M',''))) # for Mo17
    if mychr not in good_chrom:continue
    st = int(new[1])
    sp = int(new[2])
    nlist = []
    diff = abs(st-sp)+1
    mylen += diff
    a = expte_dict[mychr].find(st, sp) # check with expressed TE elements region
    if len(a) == 0:continue # only look for UMR inside expressed TE region
    for n in range(10):
        d1 = st - 100*n - 1
        d2 = st - 100*(n+1)
        a = umr_dict[mychr].find(d2, d1)
        if len(a) > 0:
            name = str(-100*(n+1)) + ':UMR|' + str(xx) + '=NA'
            out.write(mychr+'\t'+str(d2)+'\t'+str(d1)+'\t'+name+'\n')
        else:
            name = str(-100*(n+1)) + ':UMR|' + str(xx)
            out.write(mychr+'\t'+str(d2)+'\t'+str(d1)+'\t'+name+'\n')
                    
    name = str(0) + ':UMR|' + str(xx)
    out.write(mychr+'\t'+str(st)+'\t'+str(sp)+'\t'+name+'\n')
        
    for n in range(10):
        d1 = sp + 100*n + 1
        d2 = sp + 100*(n+1)
        a = umr_dict[mychr].find(d1, d2)
        if len(a) > 0:
            name = str(100*(n+1)) + ':UMR|' + str(xx) + '=NA'
            out.write(mychr+'\t'+str(d1)+'\t'+str(d2)+'\t'+name+'\n')
        else:
            name = str(100*(n+1)) + ':UMR|' + str(xx)
            out.write(mychr+'\t'+str(d1)+'\t'+str(d2)+'\t'+name+'\n')
    xx += 1
    
umr.close()
out.close()

print (diff)

301
